<h1>High resolution cross-correlation spectroscopy of KELT-9 b</h1>
Welcome to this tutorial on high-resolution cross-correlation spectroscopy of exoplanets.
<br>
This tutorial aims to introduce you to the basic concepts and steps needed to understand and carry out typical analyses of high-resolution exoplanet spectra. We will use an observation of two transits of the exoplanet KELT-9 b as an example, and construct an analysis cascade that produces a detection of gas-phase atomic iron in the exoplanet atmosphere using cross-correlation, starting from the pipeline products produced by the data reduction software of HARPS-N.
<br>

Although focussed on transit transmission spectroscopy in the optical, these concepts also apply to day-side emission spectra and infrared wavelengths. The approach generally follows the steps taken in [Hoeijmakers et al. 2019](www.aanda.org/articles/aa/pdf/2019/07/aa35089-19.pdf) (KELT-9 b) and in particular [Hoeijmakers et al. 2020](www.aanda.org/articles/aa/pdf/2020/09/aa38365-20.pdf) (WASP-121 b).
<!-- <br>
<br>
<br>
<br> -->



<h2>We'll make use of:</h2>

* Spectral orders extracted and processed by the HARPS-N pipeline (e2ds FITS files).
* Wavelength solutions provided by the pipeline, encoded in the FITS headers.
* The observing dates of the time series, corrected for light-travel delay using the RA and DEC coordinates.
* The line-of-sight velocity of Earth with respect to the barycenter during the time series.
* The orbital parameters, to determine the orbital velocity and orbital phase at each time in the time-series.
* Standard python packages and Astropy.
* Three convenience functions defined below.
* Model spectra templates of ionised iron and the spectral continuum.

<br>



<br>
<br>
<br>
<br>

![title](gfx/graph.png)

<h2>The cross-correlation function:</h2>

Cross correlation has been used in various flavours since the first application in [Snellen et al. 2010](https://www.nature.com/articles/nature09111). In this example, we use the simple weighted average:
<br>
<br>
<br>
<br>
$$
C(v,t) = \frac{\sum_i^N x_i(t) T_i(v)}{\sum_i^N T_i(v)}
$$
<br>
Here, $T$ is the template shifted to a radial velocity $v$. $x$ is the spectrum of the planet, observed at time $t$. $T$ and $x$ are necessarily defined onto the same grid. $C$ then is a 2-dimensional matrix, as function of time along the transit and the radial velocity shift of the template. At the correct combination of time and velocity, the signal of the planet should appear.

<br>
<br>



For example, here is the 2D CCF of metals in KELT-9 b from [Hoeijmakers et al. 2019](https://www.aanda.org/articles/aa/pdf/2019/07/aa35089-19.pdf):

![title](gfx/ccf.png)

<br>
<br>
<br>
<br>
<br>
<br>

<h2>The KELT-9 data:</h2>

The spectrum of KELT-9 was observed during a transit in 2017 with HARPS-N. The pipeline produces multiple types of dataproducts, two of which are the 1D stitched spectra (S1D) and the extracted echelle orders (E2DS). The S1D is shown below, revealing the smooth visible spectrum of this star, which is a fast-rotating A-star, with few and broad spectral lines. The planet spectrum is embedded in this during the transit.
<br>
<br>
![title](gfx/spectrum.png)

In [ ]:
#Import statements
import astropy.io.fits as fits
from astropy.time import Time
import astropy.constants as const
from astropy import units as u, coordinates as coord
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d
def read_wave_from_header(h):
    #This reads the wavelength axis of an e2ds file, encoded as a 3rd degree polynomial.
    npx = h['NAXIS1']
    no = h['NAXIS2']
    x = np.arange(npx, dtype=float)
    wave=np.zeros((npx,no))
    coeffkeyword = 'TNG'
    key_counter = 0
    for i in range(no):
        l = x*0.0
        for j in range(4):
            l += h['TNG DRS CAL TH COEFF LL%s' %key_counter]*x**j
            key_counter +=1
        wave[:,i] = l
    wave = wave.T
    return(wave)


def mjd_to_bjd(d,RA,DEC):
    #Convert MJD to BJD to account for light travel time. Adopted from Astropy manual.
    t = Time(d,format='mjd',scale='tdb',location=coord.EarthLocation.from_geodetic(0,0,0))
    target = coord.SkyCoord(RA,DEC,unit=(u.hourangle, u.deg), frame='icrs')
    ltt_bary = t.light_travel_time(target)
    return t.tdb + ltt_bary # = BJD


def airtovac(wlnm):
    #Convert wavelengths (nm) in air to wavelengths in vaccuum (empirical).
    wlA=wlnm*10.0
    s = 1e4 / wlA
    n = 1 + (0.00008336624212083 + 0.02408926869968 / (130.1065924522 - s**2) +
    0.0001599740894897 / (38.92568793293 - s**2))
    return(wlA*n/10.0)


<br><br><br><br><br><br><br><br><br><br>
<h2>Step 1: Reading and understanding HARPS-N DRS data.</h2>
The data is located in a subdirectory called `e2ds`, which contains a sequence of FITS files corresponding to the exposures taken by the instrument. We read these using the astropy.io.fits module, as well as the header and the wavelength solution.

In [ ]:
#Open an E2Ds file.

filelist = os.listdir('e2ds')

with fits.open('e2ds/'+filelist[0]) as file:
    e2ds = file[0].data
    hdr = file[0].header
    wave = read_wave_from_header(hdr)/10.0
    
    
# plt.figure(figsize=(17,5))    
# plt.imshow(e2ds,aspect='auto')

  
plt.figure(figsize=(17,5)) 
plt.plot(wave[53],e2ds[53])
plt.show()

In [ ]:
#Plot orders 40-69 in one plot. Notice blaze and overlap.


plt.figure(figsize=(17,5)) 
for i in range(40,69):
    plt.plot(wave[i],e2ds[i])
plt.show()

In [ ]:
#Read all e2ds files, their wavelengths and their MJD dates into lists.
list_of_e2ds = []
list_of_waves2d = []
MJD = []
BERV = []

for f in filelist:
    with fits.open('e2ds/'+f) as file:
        list_of_e2ds.append(file[0].data)
        hdr = file[0].header
        list_of_waves2d.append(airtovac(read_wave_from_header(hdr)/10.0))
        MJD.append(hdr['MJD-OBS'])
        BERV.append(hdr['HIERARCH TNG DRS BERV'])
        

In [ ]:
#Plot some e2ds files.

plt.figure(figsize=(17,5)) 
plt.plot(list_of_waves2d[10][68],list_of_e2ds[10][68])
plt.plot(list_of_waves2d[20][68],list_of_e2ds[20][68])
plt.plot(list_of_waves2d[30][68],list_of_e2ds[30][68])
plt.plot(list_of_waves2d[40][68],list_of_e2ds[40][68])
plt.show()

<br><br><br><br><br><br><br><br><br><br>
<h2>Step 2: Sorting, normalising and reformatting the E2DS data, shifting to vaccuum wavelengths.</h2>
<br>
We wish to sort the spectra in time, normalise out flux differences, and also to reformat our lists so that we work with lists of spectral orders rather than a list of files.
<br><br><br>

![title](gfx/reformatting.png)

In [ ]:
#Sort the spectra in time.

sorting = np.argsort(MJD)
# print(sorting)
MJD = np.array(MJD)[sorting]
BERV = np.array(BERV)[sorting]
list_of_waves2d = np.array(list_of_waves2d)[sorting]
list_of_e2ds = np.array(list_of_e2ds)[sorting]

In [ ]:
print(MJD)

In [ ]:
#Reformat into a list of spectral orders rather than a list of exposures.
list_of_orders = []
list_of_waves = []
N_exp = len(filelist)


for i in range(0,69):
    order = np.zeros((N_exp,4096))
    for j in range(0,N_exp):
        order[j,:] = list_of_e2ds[j][i]
    list_of_orders.append(order)
    list_of_waves.append(list_of_waves2d[0][i])

    
plt.figure(figsize=(17,5))
plt.imshow(list_of_orders[68],aspect='auto')
#Make sure airtovac is also applied.

In [ ]:
#Normalise each spectral order to its mean flux.
list_of_orders_norm=[]



for i in range(len(list_of_orders)):
    order =list_of_orders[i]
    order_norm = (order.T / np.mean(order,axis=1)).T
    list_of_orders_norm.append(order_norm)





plt.figure(figsize=(17,5))
plt.imshow(list_of_orders_norm[68],aspect='auto')




<h3>Be careful: This normalisation has normalised the mean flux of all orders to 1.0. This is done mainly for plotting purposes (otherwise each row in the CCF would be scaled and you wouldn't see much). In reality, you can perform this normalisation at the CCF stage. That would correctly take into account the fact that blue orders receive less flux than redder orders. <br>
<br>
To correct the spectra still better, other normalisation or detrending can be done here at this stage, as well.</h3>

<br><br><br><br><br><br><br><br><br><br>
<h2>Step 3: Reading in a cross-correlation template.</h2>
Model spectra of the exoplanet transmission spectrum are provided in the `templates` directories as 2-row FITS images. The first row corresponds to wavelength, the second to flux. We continuum normalise these so that they can be used as cross-correlation templates.

In [ ]:
#Read the template and continuum-normalise
TP = fits.getdata('templates/4000K_1_Fe_p.fits')
CO = fits.getdata('templates/4000K_1_B_2p.fits')
wlt = TP[0]
fxt = TP[1]-CO[1]



plt.figure(figsize=(17,5))
plt.plot(wlt,fxt)

<br><br><br><br><br><br><br><br><br><br>
<h2>Step 4: Cross-correlation</h2>

<br>
<br>
We now have everything in hand to start doing the cross-correlation. Recall the formula for the weighted average that we will be applying:<br>
$$
C(v,t) = \frac{\sum_i^N x_i(t) T_i(v)}{\sum_i^N T_i(v)}
$$

This means we will need to shift the template, interpolate it onto the wavelength/sampling grid of the data, and multiply. And we do that for each of the spectra in the time series, for a range of velocity steps to build up the 2D CCF.

In [ ]:
#Demonstrate shifting:
beta = 1 - 5.0 *u.km/u.s / const.c

wlt_shifted = wlt*beta

plt.plot(wlt,fxt)
plt.plot(wlt_shifted,fxt)
plt.xlim(300,301)

In [ ]:
#Show that interpolation works.

fxt_i =interp1d(wlt*beta,fxt)(list_of_waves[10])
plt.figure(figsize=(17,5))
plt.plot(wlt,fxt)
plt.plot(list_of_waves[10],fxt_i,'.-',alpha=0.5,color='red')
plt.xlim(417.99,418.01)
plt.ylim((-0.002,0.001))

In [ ]:
#Define axes: Radial velocity, the CCF matrix, and reformatting the data into arrays.

RVt = np.arange(-100,101,1)

CCF = np.zeros((N_exp,len(RVt)))

data = np.hstack(list_of_orders_norm)
wave = np.hstack(list_of_waves)


plt.figure(figsize=(17,5))
plt.plot(wave)
plt.xlim(0,50000)
plt.ylim(380,440)

In [ ]:
#The cross-correlation (slow version)


for i in range(len(RVt)):
    rv = RVt[i]
    beta = 1+rv*u.km/u.s /const.c
    fxt_i = interp1d(wlt*beta,fxt)(wave)
    for j in range(N_exp):
        x = data[j]
        CCF[j,i] = np.sum(x*fxt_i)/np.sum(fxt_i)



<br><br><br><br><br><br><br><br><br><br>
<h2>Step 5: Time-differential and visualising the planet atmosphere</h2>
Now we proceed with doing the time-differential, which is dividing the CCF by its time-average. This should make the signature of the planet very clear.

In [ ]:
plt.figure(figsize=(17,5))
plt.imshow(CCF/np.mean(CCF,axis=0),aspect='auto',origin='lower')
plt.xlabel('Radial velocity (km/s)')
plt.ylabel('Exposure number')

<br><br><br><br><br><br><br><br><br><br>
<h2>Step 6: Calculating the orbital phase and velocity of the planet at each time.</h2>
To confirm that the signature we're seeing is co-moving with the planet, we compute and overplot our expectation of the planet's velocity using its known orbital parameters. We compute the orbital phase and derive the radial velocity at each time of our observations.

In [ ]:
#Use the equation of the circumference of a circle to calculate the orbital velocity.
P=1.4811235  #days
a=0.03462    #AU
vsys=-18     #km/s
RA='20:31:26.4'
DEC='+39:56:20'
Tc=Time(2457095.68572,format='jd',scale='tdb')
inclination=86.79 #deg

v_orb = (2*np.pi*a*u.AU / (P*u.d)).to(u.km/u.s)
print(v_orb)

In [ ]:
#Correct MJD to BJD, calculate the time elapsed since T_c and obtain orbital phase at each time. 

phase = (mjd_to_bjd(MJD,RA,DEC)-Tc).value/P

In [ ]:
#Derive the instantaneous radial velocity at which the planet is expected to be.
RV = np.sin(2*np.pi*phase)*v_orb*np.sin(np.radians(inclination))+vsys*u.km/u.s-BERV*u.km/u.s

In [ ]:
#Overplot planet velocity:


plt.figure(figsize=(17,5))
plt.imshow(CCF/np.mean(CCF,axis=0),aspect='auto',origin='lower',extent=[np.min(RVt),np.max(RVt),0,N_exp-1])
plt.plot(RV,np.arange(0,N_exp),'--',color='red')
plt.xlim(-100,100)
plt.xlabel('Radial velocity (km/s)')
plt.ylabel('Exposure number')